In [0]:
import tensorflow as tf
import random
from tensorflow.examples.tutorials.mnist import input_data


In [0]:
tf.reset_default_graph()

In [14]:
tf.set_random_seed(777)
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

learning_rate = 0.001
training_epochs = 5
batch_size = 100

X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
# 컨볼루젼 레이어 쌓는것(중요)
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev = 0.01))
L1 = tf.nn.conv2d(X_img, W1, strides = [1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')

W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev = 0.01))
L2 = tf.nn.conv2d(L1, W2, strides = [1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
L2_flat = tf.reshape(L2, [-1,7*7*64])

W3 = tf.get_variable("W3", shape=[7*7*64, 10], initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))

logits = tf.matmul(L2_flat, W3) + b
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('Learning started. It takes sometime.') 
for epoch in range(training_epochs) : # 미니배치
  avg_cost = 0
  total_batch = int(mnist.train.num_examples / batch_size) # 토탈배치 55000/100 = 550번 가중치 갱신
  
  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    feed_dict = {X: batch_xs, Y: batch_ys}
    c, _ = sess.run([cost, optimizer], feed_dict = feed_dict) # 딕셔너리만 참조할 수 있음 sess.run()
    avg_cost += c/total_batch # 평균 비용을 구함/토탈배치의
  print('epoch:','%04d' % (epoch + 1), 'cost = ', '{:.9f}'.format(avg_cost))
print('Learning Finished')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy:", sess.run(accuracy, feed_dict ={X:mnist.test.images, Y:mnist.test.labels}))

# Get one predict
r = random.randint(0, mnist.test.num_examples - 1)
print("Label : ", sess.run(tf.argmax(mnist.test.labels[r:r+1], 1)))
print("Prediction:", sess.run(tf.argmax(logits, 1), feed_dict = {X: mnist.test.images[r:r+1]}))

sess.close()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning started. It takes sometime.
epoch: 0001 cost =  0.004471336
epoch: 0001 cost =  0.009355842
epoch: 0001 cost =  0.013953050
epoch: 0001 cost =  0.018517955
epoch: 0001 cost =  0.023285931
epoch: 0001 cost =  0.027897219
epoch: 0001 cost =  0.032653731
epoch: 0001 cost =  0.037112175
epoch: 0001 cost =  0.041563934
epoch: 0001 cost =  0.045809096
epoch: 0001 cost =  0.050077082
epoch: 0001 cost =  0.054313869
epoch: 0001 cost =  0.058505479
epoch: 0001 cost =  0.062569520
epoch: 0001 cost =  0.066651526
epoch: 0001 cost =  0.070667023
epoch: 0001 cost =  0.074705550
epoch: 0001 cost =  0.078610843
epoch: 0001 cost =  0.082385964
epoch: 0001 cost =  0.086155050
epoch: 0001 cost =  0.089831161
epoch: 0001 cost =  0.093578472
epoch: 0001 cost =  0.097080552
epoch: 0001 cost =  0.100555459
e

In [8]:
# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy:", sess.run(accuracy, feed_dict ={X:mnist.test.images, Y:mnist.test.labels}))

# Get one predict
r = random.randint(0, mnist.test.num_examples - 1)
print("Label : ", sess.run(tf.argmax(mnist.test.labels[r:r+1], 1)))
print("Prediction:", sess.run(tf.argmax(logits, 1), feed_dict = {X: mnist.test.images[r:r+1]}))

sess.close()

Accuracy: 0.9853
Label :  [2]
Prediction: [2]
